*Credit: Best for Britain*

## Executive summary

Today, the Office for Budget Responsibility releases its Spring 2025 Economic and Fiscal Outlook. The OBR is an independent body that produces economic and fiscal forecasts for the UK government.

* The OBR forecasts that income sources will rise across all categories (employment: +£24-29B, self-employment: +£12-25B, non-labor: +£19-43B annually, CPI: +0.3-0.9 annually).
* We project absolute poverty rates will fall (0.2-0.7 percentage points), while income inequality will widen.
* We project that households in the lowest decile will see net incomes 1.9% higher in 2030 under the new forecast than the Autumn 2024 forecast, and households in the highest decile will see net incomes 3.1% higher.
* We project that the average household will have a net income £1,160 (2.5%) higher in 2030 under the new forecast than the Autumn 2024 forecast.
* We project net benefit recipient households will reduce by 0.3-0.5 points (253,000-462,000 fewer households annually).

In this report, we'll integrate the OBR's new Spring forecast into PolicyEngine's tax-benefit microsimulation model, and examine the impact this new forecast (against the Autumn 2024 EFO) on the UK household sector. All impacts below are the impact of this forecast change *alone*, excluding any policy measures not already in the baseline policy.

In [1]:
from policyengine import Simulation


# Autumn 2024 OBR Forecast
AUTUMN_24_OBR_FORECAST = {
    "gov.obr.employment_income": {
        "year:2025:1": 1215.6,
        "year:2026:1": 1246.5,
        "year:2027:1": 1277.7,
        "year:2028:1": 1312.9,
        "year:2029:1": 1352.9,
    },
    "gov.obr.mixed_income": {
        "year:2025:1": 177.6,
        "year:2026:1": 196.7,
        "year:2027:1": 205.3,
        "year:2028:1": 214.6,
        "year:2029:1": 225.0,
    },
    "gov.obr.non_labour_income": {
        "year:2025:1": 460.8,
        "year:2026:1": 512.7,
        "year:2027:1": 535.0,
        "year:2028:1": 554.0,
        "year:2029:1": 571.8,
    },
    "gov.obr.consumer_price_index": {
        "year:2025:1": 138.1,
        "year:2026:1": 141.1,
        "year:2027:1": 144.1,
        "year:2028:1": 147.1,
        "year:2029:1": 150.1,
    },
}

# Spring 2025 OBR Forecast - Default to Autumn 2024 values, update these when the new forecast is released
SPRING_25_OBR_FORECAST = {
    "gov.obr.employment_income": {
        "year:2025:1": 1239.8,
        "year:2026:1": 1272.5,
        "year:2027:1": 1305.3,
        "year:2028:1": 1340.7,
        "year:2029:1": 1382.0,
    },
    "gov.obr.mixed_income": {
        "year:2025:1": 202.8,
        "year:2026:1": 210.6,
        "year:2027:1": 218.7,
        "year:2028:1": 227.4,
        "year:2029:1": 237.2,
    },
    "gov.obr.non_labour_income": {
        "year:2025:1": 503.3,
        "year:2026:1": 531.6,
        "year:2027:1": 555.6,
        "year:2028:1": 574.6,
        "year:2029:1": 594.5,
    },
    "gov.obr.consumer_price_index": {
        "year:2025:1": 139.0,
        "year:2026:1": 141.7,
        "year:2027:1": 144.5,
        "year:2028:1": 147.4,
        "year:2029:1": 150.4,
    },
}
sim = Simulation(
    country="uk",
    scope="macro",
    baseline=AUTUMN_24_OBR_FORECAST,
    reform=SPRING_25_OBR_FORECAST,
)

from policyengine_core.reforms import Reform
from IPython.display import Markdown
import pandas as pd

baseline = Reform.from_dict(AUTUMN_24_OBR_FORECAST, country_id="uk")
reform = Reform.from_dict(SPRING_25_OBR_FORECAST, country_id="uk")

link = f"https://policyengine.org/uk/policy?reform={reform.api_id}&baseline={baseline.api_id}&region=uk&timePeriod=2029"

Markdown(f"[See the forecast change in PolicyEngine]({link})")

/Users/nikhilwoodruff/policyengine/obr-forecast/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[See the forecast change in PolicyEngine](https://policyengine.org/uk/policy?reform=80230&baseline=80227&region=uk&timePeriod=2029)

## The forecast change

In this report we'll focus on the changes to core economic variables: employment income, mixed income, non-labour income, and inflation. The tables below show the prior and new values for these variables (totals, £ billion).

* The **employment income** forecast by the OBR is used to update wages and salaries in PolicyEngine for future year modelling.
* The **mixed income** forecast by the OBR is used to update self-employment incomes.[^1]
* The **non-labour income** forecast by the OBR is used to update income from savings, investments, and pensions.
* The **inflation** forecast by the OBR is used to update the price level in PolicyEngine, including policy parameters that are indexed to inflation.

[^1]: Per the [OBR's definition](https://obr.uk/forecasts-in-depth/the-economy-forecast/income/#disposable), *"mixed income is largely composed of self-employment income, which is derived from our forecasts of self-employment and whole economy average earnings"*.

In [28]:
# Create a table for the baseline and reform, column 1 is the baseline, column 2 is the reform, column 3 is the change. Years are rows

def add_thousands_separator(x):
    return "{:,}".format(x)

def create_table(baseline, reform):
    years = []
    autumn_values = []
    spring_values = []
    change_values = []
    variables = []
    key_to_label = {
        "gov.obr.employment_income": "Employment income",
        "gov.obr.mixed_income": "Mixed income",
        "gov.obr.non_labour_income": "Non-labour income",
        "gov.obr.consumer_price_index": "Consumer Price Index",
    }

    for year in range(2025, 2030):
        for key in baseline.keys():
            years.append(year)
            variables.append(key_to_label[key])
            autumn_values.append(baseline[key][f"year:{year}:1"])
            spring_values.append(reform[key][f"year:{year}:1"])
            if key != "gov.obr.consumer_price_index":
                change_values.append(reform[key][f"year:{year}:1"] - baseline[key][f"year:{year}:1"])
            else:
                change_values.append((reform[key][f"year:{year}:1"] / baseline[key][f"year:{year}:1"] - 1) * 100)
    
    change_label = "Change (%)" if key == "gov.obr.consumer_price_index" else "Change"
    df = pd.DataFrame({
        "Year": years,
        "Variable": variables,
        "Autumn 2024": autumn_values,
        "Spring 2025": spring_values,
        change_label: change_values,
    })
    df["Year"] = df["Year"].astype(str)
    df["Autumn 2024"] = df["Autumn 2024"].round(1)
    df["Spring 2025"] = df["Spring 2025"].round(1)
    df[change_label] = df[change_label].round(1)

    df["Autumn 2024"] = df["Autumn 2024"].apply(add_thousands_separator)
    df["Spring 2025"] = df["Spring 2025"].apply(add_thousands_separator)
    df[change_label] = df[change_label].apply(add_thousands_separator)

    return df

reform_table = create_table(AUTUMN_24_OBR_FORECAST, SPRING_25_OBR_FORECAST)

def get_table(key):
    return Markdown(f"### {key}\n\n" + reform_table[reform_table.Variable == key].drop("Variable", axis=1).set_index("Year").to_markdown())

get_table("Employment income")

### Employment income

|   Year | Autumn 2024   | Spring 2025   |   Change (%) |
|-------:|:--------------|:--------------|-------------:|
|   2025 | 1,215.6       | 1,239.8       |         24.2 |
|   2026 | 1,246.5       | 1,272.5       |         26   |
|   2027 | 1,277.7       | 1,305.3       |         27.6 |
|   2028 | 1,312.9       | 1,340.7       |         27.8 |
|   2029 | 1,352.9       | 1,382.0       |         29.1 |

In [3]:
get_table("Mixed income")

### Mixed income

|   Year |   Autumn 2024 |   Spring 2025 |   Change |
|-------:|--------------:|--------------:|---------:|
|   2025 |         177.6 |         202.8 |     25.2 |
|   2026 |         196.7 |         210.6 |     13.9 |
|   2027 |         205.3 |         218.7 |     13.4 |
|   2028 |         214.6 |         227.4 |     12.8 |
|   2029 |         225   |         237.2 |     12.2 |

In [4]:
get_table("Non-labour income")

### Non-labour income

|   Year |   Autumn 2024 |   Spring 2025 |   Change |
|-------:|--------------:|--------------:|---------:|
|   2025 |         460.8 |         503.3 |     42.5 |
|   2026 |         512.7 |         531.6 |     18.9 |
|   2027 |         535   |         555.6 |     20.6 |
|   2028 |         554   |         574.6 |     20.6 |
|   2029 |         571.8 |         594.5 |     22.7 |

In [29]:
get_table("Consumer Price Index")

### Consumer Price Index

|   Year |   Autumn 2024 |   Spring 2025 |   Change (%) |
|-------:|--------------:|--------------:|-------------:|
|   2025 |         138.1 |         139   |          0.7 |
|   2026 |         141.1 |         141.7 |          0.4 |
|   2027 |         144.1 |         144.5 |          0.3 |
|   2028 |         147.1 |         147.4 |          0.2 |
|   2029 |         150.1 |         150.4 |          0.2 |

## Public sector net worth

The table below shows the change in public sector net worth as a result of the forecast change (change to government tax revenue forecast minus spending). Public sector net worth (PSNW), defined by the [OBR](https://obr.uk/box/broader-measures-of-the-public-sector-balance-sheet/), is the broadest National Accounts measure of the public sector balance sheet in the UK.

In the table below, we report *only the change to PSNW implied by the PolicyEngine model*, which is the simulated change in tax revenues less government spending on cash benefits implied by the change in the OBR forecast on the four economic metrics above (excluding debt interest spending). We do not account for behavioural responses in this score.

Table 1: Change in public sector net worth (£ billion)

In [6]:
# First, PSND from forecast changes
psnd_change = []
years = []
for year in range(2025, 2030):
    psnd_baseline = sim.baseline_simulation.calculate("gov_balance", year).sum()/1e9
    psnd_reform = sim.reform_simulation.calculate("gov_balance", year).sum()/1e9
    change = psnd_reform - psnd_baseline
    psnd_change.append(round(change, 1))
    years.append(year)

years.append("2025-2029")
psnd_change.append(round(sum(psnd_change), 1))

Markdown(pd.DataFrame({"Year": years, "PSNW change (£ billions)": psnd_change}).set_index("Year").T.to_markdown())

|                          |   2025 |   2026 |   2027 |   2028 |   2029 |   2025-2029 |
|:-------------------------|-------:|-------:|-------:|-------:|-------:|------------:|
| PSNW change (£ billions) |   28.4 |   19.3 |   20.8 |   20.9 |   21.5 |       110.9 |

In [7]:
Markdown(f"As shown, the OBR forecast update raises the government budget by £{psnd_change[-1]} billion over the period 2025-2029. Per year, this is on average £{round(psnd_change[-1]/5, 1)} billion.")

As shown, the OBR forecast update raises the government budget by £110.9 billion over the period 2025-2029. Per year, this is on average £22.2 billion.

## Poverty rates

The government reports annually on four poverty metrics:

* **Absolute poverty before housing costs**: households with income below 60% of the 2010/11 median income, adjusted for inflation.
* **Absolute poverty after housing costs**: households with income below 60% of the 2010/11 median income after housing costs, adjusted for inflation.
* **Relative poverty before housing costs**: households with income below 60% of the median income in the current year.
* **Relative poverty after housing costs**: households with income below 60% of the median incomeafter housing costs in the current year .

For each of these metrics, we simulate the change in the poverty rate and headcount as a result of the forecast change.

In [8]:
def get_poverty(simulation, year, absolute, bhc, rate):
    population = simulation.calculate("person_id", year).count()
    if absolute:
        if bhc:
            in_poverty = simulation.calculate("in_poverty", year, map_to="person").sum()
        else:
            in_poverty = simulation.calculate("in_poverty_ahc", year, map_to="person").sum()
    else:
        if bhc:
            in_poverty = simulation.calculate("in_relative_poverty_bhc", year, map_to="person").sum()
        else:
            in_poverty = simulation.calculate("in_relative_poverty_ahc", year, map_to="person").sum()

    if rate:
        return round(in_poverty/population * 100, 1)

    else:
        return round(in_poverty / 1e3)


In [9]:
def get_rate_table(absolute, bhc, i):
    years = []
    poverty_rates_autumn = []
    headcounts_autumn = []
    poverty_rates_spring = []
    headcounts_spring = []
    change_rates = []
    change_headcounts = []
    for year in range(2025, 2030):
        years.append(year)
        poverty_rates_autumn.append(get_poverty(sim.baseline_simulation, year, absolute, bhc, rate=True))
        poverty_rates_spring.append(get_poverty(sim.reform_simulation, year, absolute, bhc, rate=True))
        headcounts_autumn.append(get_poverty(sim.baseline_simulation, year, absolute, bhc, rate=False))
        headcounts_spring.append(get_poverty(sim.reform_simulation, year, absolute, bhc, rate=False))
        change_rates.append(round(poverty_rates_spring[-1] - poverty_rates_autumn[-1], 1))
        change_headcounts.append(round(headcounts_spring[-1] - headcounts_autumn[-1], 0))
    
    df = pd.DataFrame({
        "Year": years,
        "Autumn EFO rate (%)": poverty_rates_autumn,
        "Autumn EFO headcount (thousands)": headcounts_autumn,
        "Spring EFO rate (%)": poverty_rates_spring,
        "Spring EFO headcount (thousands)": headcounts_spring,
        "Change in rate (pp)": change_rates,
        "Change in headcount (thousands)": change_headcounts,
    })

    for col in df.columns:
        if "thousands" in col:
            df[col] = df[col].apply(add_thousands_separator)

    title = "Absolute poverty" if absolute else "Relative poverty"
    title += " before housing costs" if bhc else " after housing costs"

    return Markdown(f"Table {i}: {title}\n\n" + df.set_index("Year").to_markdown())

get_rate_table(absolute=True, bhc=True, i=2)

Table 2: Absolute poverty before housing costs

|   Year |   Autumn EFO rate (%) | Autumn EFO headcount (thousands)   |   Spring EFO rate (%) | Spring EFO headcount (thousands)   |   Change in rate (pp) |   Change in headcount (thousands) |
|-------:|----------------------:|:-----------------------------------|----------------------:|:-----------------------------------|----------------------:|----------------------------------:|
|   2025 |                  12.5 | 9,125                              |                  12.3 | 8,961                              |                  -0.2 |                              -164 |
|   2026 |                  12.3 | 9,025                              |                  12.1 | 8,854                              |                  -0.2 |                              -171 |
|   2027 |                  12.3 | 9,113                              |                  12.1 | 8,947                              |                  -0.2 |                              -166 |
|   2028 |                  12.3 | 9,101                              |                  12.1 | 8,968                              |                  -0.2 |                              -133 |
|   2029 |                  12.4 | 9,206                              |                  12.1 | 9,048                              |                  -0.3 |                              -158 |

In [10]:
get_rate_table(absolute=True, bhc=False, i=3)

Table 3: Absolute poverty after housing costs

|   Year |   Autumn EFO rate (%) | Autumn EFO headcount (thousands)   |   Spring EFO rate (%) | Spring EFO headcount (thousands)   |   Change in rate (pp) |   Change in headcount (thousands) |
|-------:|----------------------:|:-----------------------------------|----------------------:|:-----------------------------------|----------------------:|----------------------------------:|
|   2025 |                  16.1 | 11,713                             |                  15.6 | 11,372                             |                  -0.5 |                              -341 |
|   2026 |                  16.1 | 11,777                             |                  15.4 | 11,329                             |                  -0.7 |                              -448 |
|   2027 |                  16.2 | 11,980                             |                  15.6 | 11,529                             |                  -0.6 |                              -451 |
|   2028 |                  16.3 | 12,082                             |                  15.6 | 11,607                             |                  -0.7 |                              -475 |
|   2029 |                  16.4 | 12,196                             |                  15.7 | 11,682                             |                  -0.7 |                              -514 |

In [11]:
get_rate_table(absolute=False, bhc=False, i=4)

Table 4: Relative poverty after housing costs

|   Year |   Autumn EFO rate (%) | Autumn EFO headcount (thousands)   |   Spring EFO rate (%) | Spring EFO headcount (thousands)   |   Change in rate (pp) |   Change in headcount (thousands) |
|-------:|----------------------:|:-----------------------------------|----------------------:|:-----------------------------------|----------------------:|----------------------------------:|
|   2025 |                  21.4 | 15,569                             |                  21   | 15,312                             |                  -0.4 |                              -257 |
|   2026 |                  21.2 | 15,562                             |                  21.3 | 15,633                             |                   0.1 |                                71 |
|   2027 |                  21.3 | 15,757                             |                  21.5 | 15,869                             |                   0.2 |                               112 |
|   2028 |                  21.3 | 15,821                             |                  21.1 | 15,668                             |                  -0.2 |                              -153 |
|   2029 |                  20.9 | 15,548                             |                  21   | 15,661                             |                   0.1 |                               113 |

In [12]:
get_rate_table(absolute=False, bhc=False, i=5)

Table 5: Relative poverty after housing costs

|   Year |   Autumn EFO rate (%) | Autumn EFO headcount (thousands)   |   Spring EFO rate (%) | Spring EFO headcount (thousands)   |   Change in rate (pp) |   Change in headcount (thousands) |
|-------:|----------------------:|:-----------------------------------|----------------------:|:-----------------------------------|----------------------:|----------------------------------:|
|   2025 |                  21.4 | 15,569                             |                  21   | 15,312                             |                  -0.4 |                              -257 |
|   2026 |                  21.2 | 15,562                             |                  21.3 | 15,633                             |                   0.1 |                                71 |
|   2027 |                  21.3 | 15,757                             |                  21.5 | 15,869                             |                   0.2 |                               112 |
|   2028 |                  21.3 | 15,821                             |                  21.1 | 15,668                             |                  -0.2 |                              -153 |
|   2029 |                  20.9 | 15,548                             |                  21   | 15,661                             |                   0.1 |                               113 |

## Inequality

We also project income inequality metrics in PolicyEngine's open source model, including the Gini coefficient, the share of income held by the top 10 percent, and the share of income held by the top 1 percent.

In [13]:
def get_inequality_table(metric, i):
    years = []
    autumn_values = []
    spring_values = []
    change_values = []
    change_values_pct = []

    for year in range(2025, 2030):
        if metric == "Gini coefficient":
            autumn_values.append(sim.baseline_simulation.calculate("equiv_household_net_income", year).gini() * 100)
            spring_values.append(sim.reform_simulation.calculate("equiv_household_net_income", year).gini() * 100)
        elif metric == "Top 10% share":
            autumn_values.append(sim.baseline_simulation.calculate("equiv_household_net_income", year).top_10_pct_share() * 100)
            spring_values.append(sim.reform_simulation.calculate("equiv_household_net_income", year).top_10_pct_share() * 100)
        elif metric == "Top 1% share":
            autumn_values.append(sim.baseline_simulation.calculate("equiv_household_net_income", year).top_1_pct_share() * 100)
            spring_values.append(sim.reform_simulation.calculate("equiv_household_net_income", year).top_1_pct_share() * 100)
        years.append(year)
        change_values.append(round(spring_values[-1] - autumn_values[-1], 1))
        change_values_pct.append(round((spring_values[-1] - autumn_values[-1])/autumn_values[-1] * 100, 1))

    df = pd.DataFrame({
        "Year": years,
        "Autumn EFO": autumn_values,
        "Spring EFO": spring_values,
        "Change (pp)": change_values,
        "Change (%)": change_values_pct,
    })

    df["Autumn EFO"] = df["Autumn EFO"].round(1)
    df["Spring EFO"] = df["Spring EFO"].round(1)

    return Markdown(f"Table {i}: {metric}\n\n" + df.set_index("Year").to_markdown())

get_inequality_table("Gini coefficient", i=6)


Table 6: Gini coefficient

|   Year |   Autumn EFO |   Spring EFO |   Change (pp) |   Change (%) |
|-------:|-------------:|-------------:|--------------:|-------------:|
|   2025 |         35.3 |         35.7 |           0.4 |          1   |
|   2026 |         35.5 |         35.7 |           0.2 |          0.5 |
|   2027 |         35.6 |         35.8 |           0.2 |          0.6 |
|   2028 |         35.6 |         35.8 |           0.2 |          0.5 |
|   2029 |         35.7 |         35.9 |           0.2 |          0.5 |

In [14]:
get_inequality_table("Top 10% share", i=7)

Table 7: Top 10% share

|   Year |   Autumn EFO |   Spring EFO |   Change (pp) |   Change (%) |
|-------:|-------------:|-------------:|--------------:|-------------:|
|   2025 |         27.2 |         27.3 |           0.2 |          0.6 |
|   2026 |         27.5 |         27.7 |           0.2 |          0.8 |
|   2027 |         27.5 |         27.7 |           0.2 |          0.6 |
|   2028 |         27.6 |         27.8 |           0.1 |          0.5 |
|   2029 |         27.8 |         27.8 |          -0.1 |         -0.2 |

In [15]:
get_inequality_table("Top 1% share", i=8)

Table 8: Top 1% share

|   Year |   Autumn EFO |   Spring EFO |   Change (pp) |   Change (%) |
|-------:|-------------:|-------------:|--------------:|-------------:|
|   2025 |          7.9 |          8.2 |           0.3 |          3.9 |
|   2026 |          8.2 |          8.3 |           0.1 |          1.3 |
|   2027 |          8.2 |          8.4 |           0.1 |          1.5 |
|   2028 |          8.3 |          8.4 |           0.1 |          1.5 |
|   2029 |          8.3 |          8.5 |           0.1 |          1.4 |

## Household impacts

The OBR forecast change lowers net income for some households and raises it for others, due to households having different compositions of income sources and spending. The chart below shows an extract of a thousand randomly-sampled households from our enhanced Family Resources Survey-based microdata, with the x-axis showing the household's income percentile in 2025 and the y-axis showing the change in net income in 2030 as a result of the forecast change.

Figure 1: Change in net income by income percentile

In [16]:
from policyengine.utils.charts import *

add_fonts()

In [22]:
sim.reform_simulation.calculate("real_household_net_income", 2030).mean() / sim.baseline_simulation.calculate("real_household_net_income", 2030).mean()

1.0250177071127051

In [17]:
df = pd.DataFrame()
df["weight"] = sim.baseline_simulation.calculate("household_weight", 2030).values
df["Gain"] = sim.reform_simulation.calculate("real_household_net_income", 2030).values - sim.baseline_simulation.calculate("real_household_net_income", 2030).values
df["Percentile"] = sim.baseline_simulation.calculate("real_household_net_income", 2030).percentile_rank().values

import plotly.express as px

fig = px.scatter(
    df.sample(weights="weight", n=1_000, random_state=1),
    x="Percentile",
    y="Gain",
    color_discrete_sequence=[BLUE],
    opacity=0.5,
).update_layout(
    title="OBR forecast change impact on households by percentile",
    xaxis_title="Percentile",
    yaxis_title="Change in net income (£)",
    showlegend=False,
    font=dict(family="Roboto Serif"),
    font_color="black",
    yaxis_range=[-5_000, 5_000],
    yaxis_tickformat=",.0f",
    margin={
      "l": 50,
      "r": 50,
      "b": 100,
      "t": 100,
      "pad": 4
    },
    annotations=[
      {
        "x": 1,
        "y": -0.25,
        "xref": "paper",
        "yref": "paper",
        "text": "Source: PolicyEngine UK",
        "showarrow": False,
        "font": {
          "family": "Roboto Serif",
          "size": 10,
          "color": "#616161"
        }
      }
    ],
    images=[
      {
        "source": "/logo512.png",
        "x": 1,
        "y": -0.20,
        "xref": "paper",
        "yref": "paper",
        "sizex": 0.1,
        "sizey": 0.1,
        "xanchor": "right",
        "yanchor": "bottom"
      }
    ],
)

# Add zero line

fig.add_shape(
    type="line",
    x0=0,
    y0=0,
    x1=100,
    y1=0,
    line=dict(
        color=DARK_GRAY,
        width=2,
        dash="solid",
    ),
)

Markdown(f"```plotly\n{fig.to_json()}\n```")

```plotly
{"data":[{"hovertemplate":"Percentile=%{x}\u003cbr\u003eGain=%{y}\u003cextra\u003e\u003c\u002fextra\u003e","legendgroup":"","marker":{"color":"#2C6496","opacity":0.5,"symbol":"circle"},"mode":"markers","name":"","orientation":"v","showlegend":false,"x":[40.0,80.0,35.0,22.0,16.0,42.0,13.0,92.0,14.0,77.0,80.0,25.0,17.0,42.0,44.0,10.0,61.0,75.0,40.0,10.0,100.0,89.0,40.0,16.0,45.0,69.0,27.0,12.0,18.0,55.0,88.0,89.0,93.0,65.0,93.0,21.0,75.0,99.0,88.0,84.0,57.0,34.0,10.0,29.0,76.0,81.0,68.0,20.0,67.0,83.0,35.0,88.0,36.0,47.0,95.0,39.0,48.0,34.0,8.0,60.0,26.0,57.0,84.0,21.0,41.0,6.0,81.0,50.0,47.0,41.0,65.0,80.0,81.0,85.0,84.0,69.0,51.0,32.0,73.0,12.0,91.0,9.0,91.0,49.0,43.0,28.0,55.0,34.0,58.0,25.0,97.0,10.0,53.0,46.0,92.0,44.0,93.0,60.0,14.0,54.0,81.0,8.0,35.0,89.0,29.0,42.0,14.0,61.0,9.0,45.0,47.0,69.0,79.0,84.0,23.0,73.0,99.0,60.0,82.0,50.0,17.0,74.0,96.0,40.0,6.0,47.0,26.0,19.0,95.0,46.0,51.0,92.0,7.0,95.0,32.0,12.0,90.0,36.0,14.0,81.0,21.0,91.0,23.0,73.0,40.0,15.0,71.0,55.0,39.0,7.0,42.0,90.0,60.0,36.0,3.0,38.0,34.0,10.0,37.0,48.0,58.0,43.0,83.0,30.0,92.0,45.0,29.0,8.0,47.0,52.0,21.0,26.0,87.0,52.0,53.0,58.0,91.0,10.0,89.0,4.0,100.0,68.0,73.0,98.0,50.0,67.0,90.0,22.0,24.0,70.0,56.0,64.0,83.0,57.0,84.0,98.0,87.0,45.0,45.0,89.0,11.0,3.0,46.0,38.0,65.0,18.0,42.0,34.0,64.0,38.0,23.0,89.0,55.0,44.0,2.0,31.0,77.0,44.0,89.0,81.0,6.0,1.0,62.0,59.0,31.0,35.0,46.0,73.0,12.0,67.0,35.0,25.0,100.0,6.0,5.0,53.0,79.0,45.0,22.0,5.0,89.0,27.0,35.0,12.0,9.0,4.0,23.0,8.0,91.0,39.0,38.0,3.0,12.0,48.0,85.0,95.0,99.0,83.0,94.0,94.0,70.0,76.0,30.0,54.0,13.0,16.0,26.0,3.0,41.0,12.0,5.0,76.0,99.0,87.0,31.0,85.0,83.0,47.0,6.0,47.0,38.0,22.0,68.0,1.0,61.0,35.0,99.0,48.0,77.0,89.0,17.0,55.0,24.0,38.0,100.0,82.0,63.0,54.0,26.0,100.0,69.0,61.0,17.0,66.0,21.0,24.0,34.0,74.0,100.0,23.0,94.0,42.0,67.0,67.0,74.0,51.0,49.0,53.0,51.0,63.0,49.0,36.0,83.0,41.0,8.0,58.0,45.0,43.0,55.0,51.0,1.0,64.0,35.0,45.0,35.0,63.0,80.0,25.0,46.0,29.0,95.0,50.0,78.0,84.0,3.0,16.0,67.0,85.0,90.0,66.0,91.0,86.0,96.0,83.0,87.0,65.0,81.0,73.0,23.0,54.0,42.0,78.0,68.0,25.0,59.0,91.0,46.0,17.0,19.0,39.0,10.0,38.0,72.0,47.0,69.0,3.0,20.0,51.0,61.0,36.0,10.0,85.0,3.0,73.0,83.0,80.0,39.0,53.0,8.0,50.0,18.0,47.0,63.0,44.0,8.0,59.0,54.0,82.0,96.0,89.0,32.0,70.0,30.0,84.0,85.0,32.0,96.0,79.0,64.0,58.0,51.0,100.0,58.0,37.0,66.0,78.0,17.0,80.0,70.0,41.0,93.0,43.0,64.0,46.0,53.0,87.0,95.0,72.0,88.0,78.0,58.0,4.0,90.0,49.0,54.0,5.0,15.0,85.0,11.0,97.0,42.0,86.0,85.0,47.0,34.0,1.0,85.0,29.0,51.0,76.0,37.0,11.0,31.0,50.0,78.0,12.0,37.0,21.0,49.0,4.0,60.0,32.0,43.0,68.0,94.0,74.0,79.0,100.0,4.0,65.0,35.0,76.0,69.0,23.0,64.0,39.0,72.0,46.0,6.0,87.0,90.0,4.0,51.0,12.0,50.0,45.0,7.0,67.0,100.0,47.0,38.0,12.0,25.0,96.0,85.0,86.0,45.0,92.0,84.0,54.0,35.0,32.0,93.0,85.0,98.0,88.0,15.0,92.0,54.0,12.0,47.0,33.0,66.0,91.0,18.0,3.0,77.0,87.0,63.0,75.0,8.0,85.0,52.0,63.0,70.0,84.0,25.0,97.0,92.0,82.0,56.0,66.0,36.0,87.0,54.0,40.0,20.0,7.0,77.0,60.0,36.0,24.0,46.0,94.0,55.0,48.0,14.0,6.0,42.0,78.0,17.0,49.0,1.0,11.0,86.0,53.0,99.0,12.0,95.0,89.0,98.0,67.0,71.0,51.0,60.0,87.0,70.0,72.0,45.0,53.0,75.0,13.0,59.0,99.0,59.0,18.0,92.0,83.0,53.0,27.0,80.0,86.0,82.0,25.0,6.0,63.0,100.0,29.0,71.0,86.0,60.0,69.0,97.0,95.0,26.0,72.0,49.0,94.0,46.0,29.0,71.0,2.0,47.0,62.0,58.0,98.0,5.0,51.0,44.0,89.0,94.0,35.0,30.0,35.0,85.0,41.0,82.0,14.0,29.0,49.0,65.0,31.0,79.0,52.0,98.0,38.0,49.0,62.0,93.0,52.0,73.0,34.0,56.0,64.0,61.0,81.0,69.0,74.0,73.0,100.0,35.0,38.0,32.0,6.0,30.0,77.0,43.0,85.0,38.0,52.0,89.0,42.0,44.0,82.0,5.0,61.0,72.0,92.0,63.0,87.0,78.0,65.0,81.0,39.0,66.0,47.0,27.0,91.0,74.0,98.0,44.0,68.0,58.0,51.0,38.0,66.0,73.0,72.0,96.0,77.0,89.0,6.0,50.0,52.0,36.0,71.0,93.0,12.0,86.0,30.0,50.0,72.0,81.0,36.0,70.0,76.0,24.0,82.0,71.0,17.0,97.0,36.0,95.0,1.0,95.0,64.0,34.0,80.0,1.0,6.0,56.0,50.0,61.0,29.0,95.0,80.0,60.0,63.0,64.0,88.0,97.0,98.0,96.0,50.0,37.0,68.0,11.0,68.0,34.0,78.0,74.0,82.0,49.0,81.0,52.0,4.0,40.0,50.0,54.0,90.0,12.0,42.0,92.0,53.0,50.0,44.0,100.0,78.0,60.0,41.0,50.0,29.0,98.0,24.0,66.0,47.0,43.0,60.0,66.0,37.0,100.0,1.0,35.0,100.0,71.0,54.0,4.0,51.0,91.0,17.0,67.0,50.0,53.0,73.0,71.0,45.0,43.0,8.0,10.0,33.0,45.0,46.0,25.0,76.0,45.0,56.0,84.0,24.0,79.0,46.0,85.0,14.0,53.0,24.0,71.0,94.0,34.0,93.0,55.0,35.0,79.0,70.0,47.0,2.0,47.0,83.0,59.0,73.0,11.0,44.0,97.0,60.0,13.0,24.0,40.0,91.0,14.0,3.0,95.0,38.0,78.0,75.0,10.0,46.0,1.0,49.0,79.0,79.0,74.0,48.0,25.0,82.0,64.0,90.0,52.0,47.0,92.0,98.0,83.0,77.0,3.0,79.0,36.0,34.0,13.0,45.0,88.0,89.0,99.0,61.0,97.0,34.0,35.0,14.0,54.0,69.0,38.0,83.0,61.0,64.0,60.0,5.0,3.0,51.0,17.0,3.0,34.0,6.0,46.0,41.0,36.0,90.0,55.0,50.0,43.0,13.0,44.0,61.0,6.0,40.0,58.0,50.0,65.0,49.0,45.0,54.0,88.0,95.0,49.0,40.0,23.0,20.0,4.0,83.0,89.0,25.0,38.0,96.0,97.0,9.0,36.0,78.0,74.0,12.0,83.0,86.0,26.0,2.0,58.0,70.0,70.0,100.0,5.0,23.0,1.0,34.0,54.0,4.0,1.0,10.0,3.0,58.0,89.0,69.0,97.0,90.0,91.0,12.0,87.0,12.0,72.0,17.0,80.0,51.0,83.0,40.0,72.0,83.0,74.0,98.0,64.0,94.0,95.0,74.0,42.0,2.0,52.0,25.0,18.0,87.0,99.0,5.0,58.0,20.0,15.0,33.0,62.0,51.0,90.0,87.0,49.0,41.0,59.0,32.0,68.0,94.0,73.0,87.0,88.0,3.0,11.0,85.0,40.0,41.0,5.0,53.0,99.0,86.0,42.0,16.0,96.0,8.0,61.0,62.0,41.0,8.0,4.0,86.0,46.0,12.0,18.0,9.0,72.0,75.0,99.0,32.0,38.0,41.0,75.0,76.0],"xaxis":"x","y":[1069.958984375,1519.14453125,336.826171875,419.408203125,348.787109375,770.21484375,233.7626953125,2061.0703125,621.4736328125,1303.796875,1346.4375,470.984375,421.134765625,796.12890625,776.958984375,725.6123046875,212.51171875,881.84765625,250.16015625,-43.9033203125,12807.0625,1682.5078125,364.548828125,86.537109375,261.92578125,1612.05859375,660.388671875,194.84375,429.208984375,897.3828125,1791.0078125,2376.4765625,2922.7109375,1453.78515625,2088.515625,487.60546875,1430.5625,7533.65625,1453.8828125,2163.21875,1150.8828125,667.88671875,218.6005859375,63.064453125,1813.15234375,2340.9609375,1557.3359375,656.021484375,1390.69921875,1550.0,141.236328125,1880.3671875,883.5,935.55859375,2474.875,257.458984375,891.7890625,604.9296875,25.822265625,1052.69921875,207.646484375,1205.8984375,1651.59375,429.302734375,451.271484375,298.9619140625,1685.27734375,681.62890625,814.5859375,1537.37109375,1471.94140625,1371.1640625,1354.59765625,1628.0625,1636.9296875,174.47265625,79.72265625,720.326171875,1983.19921875,133.4248046875,1790.734375,307.0947265625,2128.4609375,684.6875,104.486328125,693.541015625,99.00390625,344.16796875,831.16015625,546.216796875,3061.125,305.17578125,878.2109375,1246.63671875,2001.8125,967.931640625,2691.0234375,1282.06640625,511.0498046875,1105.3203125,1455.23046875,597.1298828125,746.92578125,2871.25,114.443359375,754.8046875,669.458984375,1027.23828125,376.5439453125,593.48828125,862.484375,391.09375,1524.1953125,1662.9609375,40.845703125,1728.2265625,4598.84375,900.7421875,1708.20703125,370.796875,88.30859375,1134.95703125,2910.7265625,200.697265625,184.0634765625,712.4140625,456.13671875,403.638671875,2458.78125,668.3671875,355.609375,1585.8046875,343.8935546875,2436.828125,608.1171875,278.96484375,1999.53125,802.45703125,348.95703125,2070.97265625,660.3828125,2406.5,99.505859375,1584.28515625,1923.23828125,210.1875,1202.8125,76.38671875,346.3515625,195.67578125,724.8671875,1427.8046875,235.86328125,962.357421875,-126.1552734375,850.529296875,609.2890625,261.2490234375,149.00390625,989.20703125,338.72265625,803.73046875,1585.140625,162.4296875,2062.546875,821.865234375,590.90625,358.5654296875,956.28515625,627.640625,696.0078125,651.984375,1824.6015625,441.7890625,1215.05859375,831.328125,1713.0625,494.962890625,1692.5546875,20.79296875,8098.59375,1161.25390625,1370.7421875,3121.171875,1456.58984375,1834.046875,2700.640625,99.46875,103.36328125,1843.609375,953.1640625,1386.75,1548.078125,958.625,1001.8984375,3722.3125,1820.1484375,628.12890625,1015.958984375,1542.7578125,454.90234375,-21.08837890625,122.115234375,555.533203125,1466.67578125,328.107421875,738.24609375,19.16015625,454.40625,258.89453125,600.115234375,1956.984375,286.3515625,1342.23828125,656.197021484375,794.5078125,2881.74609375,707.392578125,1915.3515625,1567.82421875,15.98828125,-7.460693359375,907.59765625,991.2265625,697.337890625,76.189453125,651.671875,2304.51953125,228.5634765625,1156.49609375,1161.873046875,409.275390625,7784.4375,212.994140625,198.4765625,1034.5,1710.41796875,111.4921875,727.2734375,-16.0029296875,2658.4296875,210.177734375,103.486328125,292.3623046875,412.529296875,34.326171875,62.349609375,885.666015625,2243.5,376.244140625,833.830078125,-104.48583984375,241.6337890625,675.265625,1911.7578125,2454.34375,4511.234375,1563.53125,2370.8671875,1915.1640625,1418.3203125,1569.1953125,544.6328125,899.1953125,360.75390625,238.505859375,527.0390625,35.30810546875,489.5703125,151.8583984375,198.7529296875,1633.62109375,4998.09375,1562.3515625,97.765625,2428.6640625,1681.4609375,1271.16796875,174.4736328125,729.578125,1148.939453125,72.767578125,1676.2578125,-38.53271484375,1095.99609375,622.94921875,3601.3125,266.484375,1543.9296875,1360.3046875,107.775390625,1167.3515625,224.408203125,497.263671875,47962.5,1706.28125,1076.5078125,554.203125,64.689453125,8212.78125,923.23828125,1100.953125,89.875,1232.73046875,195.982421875,65.302734375,645.458984375,1448.5390625,14919.1875,101.5,2278.984375,1080.302734375,971.22265625,866.6171875,1930.484375,962.609375,1316.17578125,414.546875,1201.00390625,1092.46484375,871.73828125,104.3515625,1481.6640625,737.330078125,285.064453125,209.66796875,826.865234375,1051.33984375,1307.578125,1068.1015625,4.09832763671875,257.51953125,132.15625,823.771484375,30.142578125,1182.3046875,1907.41015625,186.517578125,780.01171875,64.744140625,2275.3359375,1750.4609375,1537.69921875,1464.2890625,194.2333984375,124.890625,1401.5078125,2188.078125,1451.8515625,1124.15625,2025.7265625,2533.9375,1743.234375,2013.9921875,2213.4140625,1093.60546875,1424.8203125,1381.80078125,112.0234375,339.66796875,1106.6328125,2241.80078125,820.1328125,598.78125,581.69140625,1841.671875,766.763671875,216.23046875,651.619140625,703.689453125,278.529296875,106.51171875,1284.26953125,888.78515625,1401.74609375,8.201171875,505.4140625,1146.1796875,1399.51953125,69.953125,584.91015625,1599.3828125,194.46240234375,1283.90625,1229.46875,1494.5546875,402.4609375,1253.21875,287.2099609375,943.0078125,440.947265625,87.578125,1491.00390625,764.580078125,287.4326171875,1246.09375,715.32421875,1499.8359375,2941.1171875,2693.1171875,109.201171875,1888.5390625,66.8359375,1630.625,1631.078125,43.783203125,3977.78125,1518.1484375,1644.4453125,1178.7109375,92.1875,6187.21875,100.8359375,657.419921875,836.47265625,1828.3984375,419.455078125,1458.17578125,1247.015625,236.7578125,2376.4140625,1094.35546875,1258.51953125,856.240234375,906.15625,2023.9921875,2035.3046875,1393.6171875,1911.5859375,1403.84375,469.35546875,-6.65625,2041.0859375,1051.26171875,316.97265625,36.0068359375,72.880859375,1805.3203125,413.166015625,3071.6171875,549.107421875,2121.4765625,1700.609375,737.97265625,611.71484375,211.12890625,1646.046875,549.640625,845.49609375,1518.5234375,824.244140625,10.5810546875,213.69921875,870.20703125,1370.24609375,342.8720703125,183.744140625,52.16015625,1072.82421875,41.68994140625,1822.43359375,628.115234375,1353.869140625,2306.6171875,2493.25,1316.2265625,1621.16796875,7649.28125,25.7509765625,509.2265625,626.640625,1476.8515625,727.67578125,187.05859375,845.15625,855.966796875,604.6328125,889.556640625,374.8369140625,2412.5703125,1708.7734375,37.26806640625,1155.6640625,544.0830078125,925.421875,953.552734375,36.6845703125,482.6171875,24347.25,1060.8046875,756.54296875,81.01171875,491.396484375,2879.0859375,1717.359375,1925.4609375,805.865234375,1837.609375,2407.1796875,906.37109375,872.890625,750.412109375,2110.3984375,1425.125,3249.25,1974.7421875,433.732421875,2412.359375,1109.88671875,80.287109375,865.765625,518.763671875,1046.83984375,1956.2578125,727.919921875,-6.35498046875,1237.86328125,1921.9765625,889.66015625,1028.4375,218.8486328125,1425.578125,908.0,940.6015625,1254.62890625,1874.1484375,621.419921875,3713.7578125,1684.5546875,1648.8984375,69.22265625,980.0859375,256.44921875,1775.90625,874.00390625,847.40625,823.0390625,217.232421875,1512.62109375,851.7578125,297.169921875,105.525390625,30.02734375,1938.4921875,38.21484375,896.53515625,332.6220703125,208.1689453125,28.962890625,1539.21875,279.45703125,1159.16796875,203.00390625,546.8623046875,1848.1171875,1090.25,5850.375,50.822265625,1790.328125,1571.5546875,2808.578125,1298.0859375,1554.046875,900.00390625,916.4140625,1603.3671875,1319.015625,1637.06640625,828.568359375,748.1875,1706.7109375,253.634765625,542.62109375,2885.5625,1105.03515625,217.2890625,2197.5078125,1295.3046875,1290.7265625,39.91796875,2110.29296875,2115.3515625,1554.65625,553.212890625,116.4501953125,941.63671875,11420.21875,419.259765625,1226.46875,1942.40625,419.0,1287.16796875,3232.9375,2904.96875,69.96484375,1077.27734375,418.37109375,2183.84375,818.12109375,536.396484375,924.2734375,531.8564453125,1081.99609375,1556.22265625,939.15234375,4471.15625,173.814453125,266.20703125,246.638671875,2002.625,2316.8125,595.638671875,99.404296875,12.716796875,1885.71875,717.13671875,1504.671875,7.7744140625,724.611328125,83.09375,1261.1875,77.4140625,1716.07421875,712.28515625,3724.390625,31.623046875,1012.65625,860.84765625,2708.578125,1051.62890625,1770.8515625,15.73046875,1586.21875,1111.05078125,1132.8671875,1450.5,1387.2109375,1450.671875,1405.89453125,7419.859375,1110.73828125,451.109375,70.52734375,236.4111328125,97.208984375,1297.40625,1045.404296875,1382.0625,80.884765625,1305.07421875,1696.2734375,768.931640625,798.5546875,1883.4453125,200.921875,558.203125,129.84765625,1982.625,1714.984375,1626.515625,1523.28125,870.88671875,1855.72265625,1164.87890625,1205.515625,297.18359375,457.642578125,1723.0859375,1432.3515625,4331.09375,804.84375,1243.80859375,1046.52734375,376.84375,778.126953125,1153.92578125,1072.1796875,1611.1953125,4170.5390625,1313.25390625,2095.3828125,215.2978515625,696.875,1068.31640625,656.88671875,1711.5234375,1831.734375,573.4873046875,1969.0234375,65.125,393.98046875,1087.91796875,1801.56640625,1064.755859375,1551.13671875,597.25,56.830078125,1350.484375,1385.77734375,63.625,2731.8984375,616.21875,2864.6328125,-6.89599609375,2036.96875,1524.9921875,430.708984375,2013.41015625,-14.941650390625,503.3515625,969.67578125,865.14453125,1078.17578125,38.9921875,2105.9296875,1622.765625,1449.8203125,1371.24609375,1087.55859375,1827.921875,2290.15625,3644.0,2485.15625,892.9921875,240.998046875,1202.62109375,207.970703125,2237.7109375,625.126953125,1579.01953125,1381.84765625,1565.390625,701.91015625,1991.8671875,815.93359375,16.6728515625,886.544921875,868.7109375,1444.7734375,1979.7421875,228.4794921875,1285.9765625,2171.546875,1076.53125,914.51171875,295.48046875,8061.625,1999.69921875,1077.4765625,751.3359375,956.51953125,370.123046875,3402.953125,290.04296875,1273.3046875,330.10546875,320.7421875,8631.86328125,966.625,566.818359375,25967.875,107.24462890625,740.5234375,15094.25,1368.1796875,1149.5546875,50.671875,818.91015625,1720.984375,39.65625,1060.3125,984.12890625,1080.46875,1456.9609375,1295.0234375,481.732421875,721.40625,399.1396484375,240.8193359375,599.701171875,819.048828125,628.25390625,502.25390625,1450.71484375,357.380859375,1359.26171875,1649.515625,54.109375,1607.11328125,174.78125,1685.515625,408.96484375,708.8515625,450.126953125,1354.71875,2030.0546875,804.259765625,2152.0859375,1111.91796875,572.58984375,2007.3046875,1317.453125,839.16796875,9.5875244140625,863.078125,1617.6796875,1522.0078125,1058.96875,-11.6240234375,826.416015625,3312.0,980.65234375,298.4951171875,476.861328125,762.78125,1741.15625,50.470703125,-17.48828125,2700.6875,1002.966796875,1853.15234375,460.375,306.064453125,459.453125,-8.868408203125,913.4921875,1634.21484375,1354.4375,1322.19140625,926.046875,525.56640625,1647.984375,1488.828125,1924.75,1478.9921875,1286.01171875,2067.46875,3273.234375,1709.546875,1615.6484375,344.28662109375,1941.43359375,849.9375,626.48046875,150.083984375,919.47265625,1804.015625,1861.2421875,3970.1875,1039.3671875,3817.953125,620.5234375,97.880859375,387.34765625,2071.3828125,1548.20703125,801.16796875,1933.7734375,803.37890625,1222.87890625,1086.1953125,25.7333984375,15.8857421875,1292.0546875,90.958984375,-11.09130859375,149.919921875,315.96875,931.7109375,548.193359375,208.685546875,1813.65625,1292.98828125,932.6171875,732.8359375,471.9033203125,1037.583984375,855.6484375,171.341796875,88.302734375,1213.671875,377.48828125,1182.86328125,658.44921875,816.15625,750.63671875,2433.84375,2139.828125,48.5078125,746.484375,476.630859375,235.458984375,281.33984375,1571.390625,1901.578125,52.978515625,660.16015625,2348.7734375,2813.21875,355.0859375,737.556640625,1597.390625,957.3828125,533.0732421875,1697.4140625,2106.03125,541.97265625,132.682861328125,1051.8359375,1692.55078125,1399.26171875,22337.1875,234.3544921875,445.279296875,-137.501953125,743.169921875,936.09375,-19.16162109375,1011.0869140625,413.5537109375,181.03515625,1717.01171875,1769.15625,1363.7890625,2787.0703125,1800.7421875,2516.8671875,326.205078125,1962.1875,349.9384765625,1880.08984375,461.3984375,1109.31640625,139.328125,2123.4609375,156.798828125,2045.16796875,1300.3046875,1356.53515625,5171.5078125,1161.23828125,2140.890625,1875.3984375,2460.90234375,1117.2421875,-12.7728271484375,1051.85546875,69.5546875,438.220703125,2011.6171875,6243.78125,50.0341796875,1329.80859375,93.876953125,947.939453125,632.990234375,1120.37109375,851.609375,1834.640625,1501.6328125,327.45703125,181.03125,974.95703125,623.453125,1276.75,3261.2578125,1143.21875,1903.59375,1857.0546875,83.65673828125,353.361328125,1645.0859375,786.220703125,736.212890625,300.7578125,1361.46484375,3983.609375,1597.1015625,852.08203125,32.025390625,2150.4765625,394.1337890625,1091.390625,929.76953125,685.060546875,25.228515625,27.576171875,3199.7578125,563.984375,359.33203125,731.50390625,255.88671875,1594.26171875,1245.453125,4006.640625,99.095703125,375.181640625,747.865234375,104.75,498.28125],"yaxis":"y","type":"scatter"}],"layout":{"template":{"data":{"histogram2dcontour":[{"type":"histogram2dcontour","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"choropleth":[{"type":"choropleth","colorbar":{"outlinewidth":0,"ticks":""}}],"histogram2d":[{"type":"histogram2d","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"heatmap":[{"type":"heatmap","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"heatmapgl":[{"type":"heatmapgl","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"contourcarpet":[{"type":"contourcarpet","colorbar":{"outlinewidth":0,"ticks":""}}],"contour":[{"type":"contour","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"surface":[{"type":"surface","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"mesh3d":[{"type":"mesh3d","colorbar":{"outlinewidth":0,"ticks":""}}],"scatter":[{"fillpattern":{"fillmode":"overlay","size":10,"solidity":0.2},"type":"scatter"}],"parcoords":[{"type":"parcoords","line":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatterpolargl":[{"type":"scatterpolargl","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"bar":[{"error_x":{"color":"#2a3f5f"},"error_y":{"color":"#2a3f5f"},"marker":{"line":{"color":"#E5ECF6","width":0.5},"pattern":{"fillmode":"overlay","size":10,"solidity":0.2}},"type":"bar"}],"scattergeo":[{"type":"scattergeo","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatterpolar":[{"type":"scatterpolar","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"histogram":[{"marker":{"pattern":{"fillmode":"overlay","size":10,"solidity":0.2}},"type":"histogram"}],"scattergl":[{"type":"scattergl","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatter3d":[{"type":"scatter3d","line":{"colorbar":{"outlinewidth":0,"ticks":""}},"marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scattermapbox":[{"type":"scattermapbox","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatterternary":[{"type":"scatterternary","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scattercarpet":[{"type":"scattercarpet","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"carpet":[{"aaxis":{"endlinecolor":"#2a3f5f","gridcolor":"white","linecolor":"white","minorgridcolor":"white","startlinecolor":"#2a3f5f"},"baxis":{"endlinecolor":"#2a3f5f","gridcolor":"white","linecolor":"white","minorgridcolor":"white","startlinecolor":"#2a3f5f"},"type":"carpet"}],"table":[{"cells":{"fill":{"color":"#EBF0F8"},"line":{"color":"white"}},"header":{"fill":{"color":"#C8D4E3"},"line":{"color":"white"}},"type":"table"}],"barpolar":[{"marker":{"line":{"color":"#E5ECF6","width":0.5},"pattern":{"fillmode":"overlay","size":10,"solidity":0.2}},"type":"barpolar"}],"pie":[{"automargin":true,"type":"pie"}]},"layout":{"autotypenumbers":"strict","colorway":["#636efa","#EF553B","#00cc96","#ab63fa","#FFA15A","#19d3f3","#FF6692","#B6E880","#FF97FF","#FECB52"],"font":{"color":"#2a3f5f"},"hovermode":"closest","hoverlabel":{"align":"left"},"paper_bgcolor":"white","plot_bgcolor":"#E5ECF6","polar":{"bgcolor":"#E5ECF6","angularaxis":{"gridcolor":"white","linecolor":"white","ticks":""},"radialaxis":{"gridcolor":"white","linecolor":"white","ticks":""}},"ternary":{"bgcolor":"#E5ECF6","aaxis":{"gridcolor":"white","linecolor":"white","ticks":""},"baxis":{"gridcolor":"white","linecolor":"white","ticks":""},"caxis":{"gridcolor":"white","linecolor":"white","ticks":""}},"coloraxis":{"colorbar":{"outlinewidth":0,"ticks":""}},"colorscale":{"sequential":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]],"sequentialminus":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]],"diverging":[[0,"#8e0152"],[0.1,"#c51b7d"],[0.2,"#de77ae"],[0.3,"#f1b6da"],[0.4,"#fde0ef"],[0.5,"#f7f7f7"],[0.6,"#e6f5d0"],[0.7,"#b8e186"],[0.8,"#7fbc41"],[0.9,"#4d9221"],[1,"#276419"]]},"xaxis":{"gridcolor":"white","linecolor":"white","ticks":"","title":{"standoff":15},"zerolinecolor":"white","automargin":true,"zerolinewidth":2},"yaxis":{"gridcolor":"white","linecolor":"white","ticks":"","title":{"standoff":15},"zerolinecolor":"white","automargin":true,"zerolinewidth":2},"scene":{"xaxis":{"backgroundcolor":"#E5ECF6","gridcolor":"white","linecolor":"white","showbackground":true,"ticks":"","zerolinecolor":"white","gridwidth":2},"yaxis":{"backgroundcolor":"#E5ECF6","gridcolor":"white","linecolor":"white","showbackground":true,"ticks":"","zerolinecolor":"white","gridwidth":2},"zaxis":{"backgroundcolor":"#E5ECF6","gridcolor":"white","linecolor":"white","showbackground":true,"ticks":"","zerolinecolor":"white","gridwidth":2}},"shapedefaults":{"line":{"color":"#2a3f5f"}},"annotationdefaults":{"arrowcolor":"#2a3f5f","arrowhead":0,"arrowwidth":1},"geo":{"bgcolor":"white","landcolor":"#E5ECF6","subunitcolor":"white","showland":true,"showlakes":true,"lakecolor":"white"},"title":{"x":0.05},"mapbox":{"style":"light"}}},"xaxis":{"anchor":"y","domain":[0.0,1.0],"title":{"text":"Percentile"}},"yaxis":{"anchor":"x","domain":[0.0,1.0],"title":{"text":"Change in net income (\u00a3)"},"range":[-5000,5000],"tickformat":",.0f"},"legend":{"tracegroupgap":0},"margin":{"t":100,"l":50,"r":50,"b":100,"pad":4},"font":{"family":"Roboto Serif","color":"black"},"title":{"text":"OBR forecast change impact on households by percentile"},"showlegend":false,"annotations":[{"font":{"color":"#616161","family":"Roboto Serif","size":10},"showarrow":false,"text":"Source: PolicyEngine UK","x":1,"xref":"paper","y":-0.25,"yref":"paper"}],"images":[{"sizex":0.1,"sizey":0.1,"source":"\u002flogo512.png","x":1,"xanchor":"right","xref":"paper","y":-0.2,"yanchor":"bottom","yref":"paper"}],"shapes":[{"line":{"color":"#616161","dash":"solid","width":2},"type":"line","x0":0,"x1":100,"y0":0,"y1":0}]}}
```

## Impact by income decile

The chart below shows the relative change in income held by each 2030 household income decile, as a result of the change in the OBR forecast.

Figure 2: Change in income by income decile

In [20]:
decile = sim.baseline_simulation.calculate("household_income_decile", 2030)
income = sim.baseline_simulation.calculate("real_household_net_income", 2030)
gain = sim.reform_simulation.calculate("real_household_net_income", 2030) - sim.baseline_simulation.calculate("real_household_net_income", 2030)
aggregate_change_by_decile = gain.groupby(decile).sum()
relative_change_by_decile = gain.groupby(decile).sum() / income.groupby(decile).sum()

df = pd.DataFrame({
    "Aggregate change": aggregate_change_by_decile.values,
    "Relative change": relative_change_by_decile.values,
    "Income decile": aggregate_change_by_decile.index,
})
df = df[df["Income decile"] > 0]
df["Text"] = df["Relative change"].apply(lambda x: f"{x:.1%}")

fig = px.bar(
    df,
    x="Income decile",
    y="Relative change",
    text="Text",
    color_discrete_sequence=[BLUE],
    labels={"Aggregate change": "Change in net income (£)", "Income decile": "Income decile"},
    title="OBR forecast change impact on 2030 net income by income decile",
).update_layout(
    font=dict(family="Roboto Serif"),
    yaxis_tickformat=".0%",
    xaxis_title="Income decile",
    xaxis_tickvals=list(range(1, 11)),
    showlegend=False,
    font_color="black",
    margin={
      "l": 50,
      "r": 50,
      "b": 100,
      "t": 100,
      "pad": 4
    },
    annotations=[
      {
        "x": 1,
        "y": -0.25,
        "xref": "paper",
        "yref": "paper",
        "text": "Source: PolicyEngine UK",
        "showarrow": False,
        "font": {
          "family": "Roboto Serif",
          "size": 10,
          "color": "#616161"
        }
      }
    ],
    images=[
      {
        "source": "/logo512.png",
        "x": 1,
        "y": -0.20,
        "xref": "paper",
        "yref": "paper",
        "sizex": 0.1,
        "sizey": 0.1,
        "xanchor": "right",
        "yanchor": "bottom"
      }
    ],
)
Markdown(f"```plotly\n{fig.to_json()}\n```")

```plotly
{"data":[{"alignmentgroup":"True","hovertemplate":"Income decile=%{x}\u003cbr\u003eRelative change=%{y}\u003cbr\u003eText=%{text}\u003cextra\u003e\u003c\u002fextra\u003e","legendgroup":"","marker":{"color":"#2C6496","pattern":{"shape":""}},"name":"","offsetgroup":"","orientation":"v","showlegend":false,"text":["1.9%","2.0%","2.0%","2.3%","2.0%","2.5%","2.5%","2.6%","2.4%","3.1%"],"textposition":"auto","x":[1,2,3,4,5,6,7,8,9,10],"xaxis":"x","y":[0.019247149326981478,0.02023357590682839,0.020460483075160386,0.02286974072878526,0.019911543407832855,0.024777099660026773,0.02477875671690137,0.025885340736588137,0.024425674721593793,0.03068977979172698],"yaxis":"y","type":"bar"}],"layout":{"template":{"data":{"histogram2dcontour":[{"type":"histogram2dcontour","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"choropleth":[{"type":"choropleth","colorbar":{"outlinewidth":0,"ticks":""}}],"histogram2d":[{"type":"histogram2d","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"heatmap":[{"type":"heatmap","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"heatmapgl":[{"type":"heatmapgl","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"contourcarpet":[{"type":"contourcarpet","colorbar":{"outlinewidth":0,"ticks":""}}],"contour":[{"type":"contour","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"surface":[{"type":"surface","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"mesh3d":[{"type":"mesh3d","colorbar":{"outlinewidth":0,"ticks":""}}],"scatter":[{"fillpattern":{"fillmode":"overlay","size":10,"solidity":0.2},"type":"scatter"}],"parcoords":[{"type":"parcoords","line":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatterpolargl":[{"type":"scatterpolargl","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"bar":[{"error_x":{"color":"#2a3f5f"},"error_y":{"color":"#2a3f5f"},"marker":{"line":{"color":"#E5ECF6","width":0.5},"pattern":{"fillmode":"overlay","size":10,"solidity":0.2}},"type":"bar"}],"scattergeo":[{"type":"scattergeo","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatterpolar":[{"type":"scatterpolar","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"histogram":[{"marker":{"pattern":{"fillmode":"overlay","size":10,"solidity":0.2}},"type":"histogram"}],"scattergl":[{"type":"scattergl","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatter3d":[{"type":"scatter3d","line":{"colorbar":{"outlinewidth":0,"ticks":""}},"marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scattermapbox":[{"type":"scattermapbox","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatterternary":[{"type":"scatterternary","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scattercarpet":[{"type":"scattercarpet","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"carpet":[{"aaxis":{"endlinecolor":"#2a3f5f","gridcolor":"white","linecolor":"white","minorgridcolor":"white","startlinecolor":"#2a3f5f"},"baxis":{"endlinecolor":"#2a3f5f","gridcolor":"white","linecolor":"white","minorgridcolor":"white","startlinecolor":"#2a3f5f"},"type":"carpet"}],"table":[{"cells":{"fill":{"color":"#EBF0F8"},"line":{"color":"white"}},"header":{"fill":{"color":"#C8D4E3"},"line":{"color":"white"}},"type":"table"}],"barpolar":[{"marker":{"line":{"color":"#E5ECF6","width":0.5},"pattern":{"fillmode":"overlay","size":10,"solidity":0.2}},"type":"barpolar"}],"pie":[{"automargin":true,"type":"pie"}]},"layout":{"autotypenumbers":"strict","colorway":["#636efa","#EF553B","#00cc96","#ab63fa","#FFA15A","#19d3f3","#FF6692","#B6E880","#FF97FF","#FECB52"],"font":{"color":"#2a3f5f"},"hovermode":"closest","hoverlabel":{"align":"left"},"paper_bgcolor":"white","plot_bgcolor":"#E5ECF6","polar":{"bgcolor":"#E5ECF6","angularaxis":{"gridcolor":"white","linecolor":"white","ticks":""},"radialaxis":{"gridcolor":"white","linecolor":"white","ticks":""}},"ternary":{"bgcolor":"#E5ECF6","aaxis":{"gridcolor":"white","linecolor":"white","ticks":""},"baxis":{"gridcolor":"white","linecolor":"white","ticks":""},"caxis":{"gridcolor":"white","linecolor":"white","ticks":""}},"coloraxis":{"colorbar":{"outlinewidth":0,"ticks":""}},"colorscale":{"sequential":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]],"sequentialminus":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]],"diverging":[[0,"#8e0152"],[0.1,"#c51b7d"],[0.2,"#de77ae"],[0.3,"#f1b6da"],[0.4,"#fde0ef"],[0.5,"#f7f7f7"],[0.6,"#e6f5d0"],[0.7,"#b8e186"],[0.8,"#7fbc41"],[0.9,"#4d9221"],[1,"#276419"]]},"xaxis":{"gridcolor":"white","linecolor":"white","ticks":"","title":{"standoff":15},"zerolinecolor":"white","automargin":true,"zerolinewidth":2},"yaxis":{"gridcolor":"white","linecolor":"white","ticks":"","title":{"standoff":15},"zerolinecolor":"white","automargin":true,"zerolinewidth":2},"scene":{"xaxis":{"backgroundcolor":"#E5ECF6","gridcolor":"white","linecolor":"white","showbackground":true,"ticks":"","zerolinecolor":"white","gridwidth":2},"yaxis":{"backgroundcolor":"#E5ECF6","gridcolor":"white","linecolor":"white","showbackground":true,"ticks":"","zerolinecolor":"white","gridwidth":2},"zaxis":{"backgroundcolor":"#E5ECF6","gridcolor":"white","linecolor":"white","showbackground":true,"ticks":"","zerolinecolor":"white","gridwidth":2}},"shapedefaults":{"line":{"color":"#2a3f5f"}},"annotationdefaults":{"arrowcolor":"#2a3f5f","arrowhead":0,"arrowwidth":1},"geo":{"bgcolor":"white","landcolor":"#E5ECF6","subunitcolor":"white","showland":true,"showlakes":true,"lakecolor":"white"},"title":{"x":0.05},"mapbox":{"style":"light"}}},"xaxis":{"anchor":"y","domain":[0.0,1.0],"title":{"text":"Income decile"},"tickvals":[1,2,3,4,5,6,7,8,9,10]},"yaxis":{"anchor":"x","domain":[0.0,1.0],"title":{"text":"Relative change"},"tickformat":".0%"},"legend":{"tracegroupgap":0},"title":{"text":"OBR forecast change impact on 2030 net income by income decile"},"barmode":"relative","font":{"family":"Roboto Serif","color":"black"},"margin":{"l":50,"r":50,"b":100,"t":100,"pad":4},"showlegend":false,"annotations":[{"font":{"color":"#616161","family":"Roboto Serif","size":10},"showarrow":false,"text":"Source: PolicyEngine UK","x":1,"xref":"paper","y":-0.25,"yref":"paper"}],"images":[{"sizex":0.1,"sizey":0.1,"source":"\u002flogo512.png","x":1,"xanchor":"right","xref":"paper","y":-0.2,"yanchor":"bottom","yref":"paper"}]}}
```

## Households by beneficiary status

The ONS [reports](https://www.ons.gov.uk/peoplepopulationandcommunity/personalandhouseholdfinances/incomeandwealth/bulletins/theeffectsoftaxesandbenefitsonhouseholdincome/financialyearending2023) on its self-defined net recipient rate. This is the percentage of households that receive more in (cash and in-kind) benefits than they pay in taxes.

The table below shows the change in a similar measure, defined as the percentage of households that receive more in cash benefits than they pay in taxes, as a result of the forecast change.

In [19]:
years = []
net_recipients_rate_a = []
net_recipients_rate_s = []
net_recipients_people_a = []
net_recipients_people_s = []
change_rate = []
change_people = []

for year in range(2025, 2030):
    years.append(year)
    net_recipients_rate_a.append((sim.baseline_simulation.calculate("household_benefits", year) > sim.baseline_simulation.calculate("household_tax", year)).mean())
    net_recipients_rate_s.append((sim.reform_simulation.calculate("household_benefits", year) > sim.reform_simulation.calculate("household_tax", year)).mean())
    change_rate.append(net_recipients_rate_s[-1] - net_recipients_rate_a[-1])
    net_recipients_people_a.append((sim.baseline_simulation.calculate("household_benefits", year, map_to="person") > sim.baseline_simulation.calculate("household_tax", year, map_to="person")).sum())
    net_recipients_people_s.append((sim.reform_simulation.calculate("household_benefits", year, map_to="person") > sim.reform_simulation.calculate("household_tax", year, map_to="person")).sum())
    change_people.append(net_recipients_people_s[-1] - net_recipients_people_a[-1])
df = pd.DataFrame({
    "Year": years,
    "Net recipients rate (%) (Autumn 2024)": net_recipients_rate_a,
    "Net recipients rate (%) (Spring 2025)": net_recipients_rate_s,
    "Change in rate (pp)": change_rate,
    "Net recipients (Autumn 2024) (m)": net_recipients_people_a,
    "Net recipients (Spring 2025) (m)": net_recipients_people_s,
    "Change in net recipients (k)": change_people,
})

df["Net recipients rate (%) (Autumn 2024)"] = df["Net recipients rate (%) (Autumn 2024)"].apply(lambda x: f"{x*100:.1f}")
df["Net recipients rate (%) (Spring 2025)"] = df["Net recipients rate (%) (Spring 2025)"].apply(lambda x: f"{x*100:.1f}")
df["Change in rate (pp)"] = df["Change in rate (pp)"].apply(lambda x: f"{x*100:.1f}")

df["Net recipients (Autumn 2024) (m)"] = df["Net recipients (Autumn 2024) (m)"].apply(lambda x: f"{x/1e6:,.1f}")
df["Net recipients (Spring 2025) (m)"] = df["Net recipients (Spring 2025) (m)"].apply(lambda x: f"{x/1e6:,.1f}")
df["Change in net recipients (k)"] = df["Change in net recipients (k)"].apply(lambda x: f"{x/1e3:,.0f}")


Markdown(f"Table 9: Net recipients by forecast \n\n" + df.set_index("Year").to_markdown())

Table 9: Net recipients by forecast 

|   Year |   Net recipients rate (%) (Autumn 2024) |   Net recipients rate (%) (Spring 2025) |   Change in rate (pp) |   Net recipients (Autumn 2024) (m) |   Net recipients (Spring 2025) (m) |   Change in net recipients (k) |
|-------:|----------------------------------------:|----------------------------------------:|----------------------:|-----------------------------------:|-----------------------------------:|-------------------------------:|
|   2025 |                                    45.4 |                                    44.9 |                  -0.5 |                               30.2 |                               29.8 |                           -462 |
|   2026 |                                    45.3 |                                    44.9 |                  -0.3 |                               30.5 |                               30.2 |                           -316 |
|   2027 |                                    45.1 |                                    44.7 |                  -0.4 |                               30.6 |                               30.2 |                           -379 |
|   2028 |                                    44.9 |                                    44.6 |                  -0.3 |                               30.5 |                               30.3 |                           -253 |
|   2029 |                                    44.8 |                                    44.5 |                  -0.3 |                               30.5 |                               30.2 |                           -254 |